<a href="https://colab.research.google.com/github/blindsphynx/Diploma/blob/main/ruT5_base_conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка среды выполнения

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


[T5 paper](https://arxiv.org/abs/1910.10683)

 - Base: [HF Model API](https://huggingface.co/sberbank-ai/ruT5-base)

 [Model parameters](https://huggingface.co/transformers/model_doc/t5.html)

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split
import torch

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# raw_model = 'ai-forever/ruT5-base' 
pre_trained = 'drive/MyDrive/ruT5-base_conversational'
model = T5ForConditionalGeneration.from_pretrained(pre_trained) #.cuda();
tokenizer = T5Tokenizer.from_pretrained(pre_trained)

def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs, max_new_tokens=150)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

# Подготовка датасета для дообучения

In [ ]:
df = pd.read_csv('drive/MyDrive/dataset.csv', sep='|', encoding="utf-8")
pd.options.display.max_colwidth = 500

In [ ]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.5, random_state=1)
pairs = df_train[['q', 'a']].values.tolist()

# Файн-тьюнинг


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
batch_size = 16  
report_steps = 1500  
epochs = 10  

In [ ]:
new_model_name = 'drive/MyDrive/ruT5-base_conversational'   

model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))
            model.save_pretrained(new_model_name)
    tokenizer.save_pretrained(new_model_name)  

# Генерация

In [ ]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device) 
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs, max_new_tokens=200)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [ ]:
query = input()
print("You: ", query)
print("ruT5: ", answer(query, do_sample=True, top_p=0.9))